# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [1]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [2]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [3]:
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)#输入和输出通道数分别为1和10
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)#输入和输出通道数分别为10和20
        self.conv2_drop = nn.Dropout2d()#随机选择输入的信道，将其设为0
        self.fc1 = nn.Linear(320, 50)#输入的向量大小和输出的大小分别为320和50
        self.fc2 = nn.Linear(50, 10)
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))#conv->max_pool->relu
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))#conv->dropout->max_pool->relu
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))#fc->relu
        x = F.dropout(x, training=self.training)#dropout
        x = self.fc2(x)
        return F.log_softmax(x,dim=0)
    
model = SimpleNet()

# TODO:define loss function and optimiter
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
print (len(train_dataset))

60000


Next, we can start to train and evaluate!

In [5]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        # wrap them in Variable
        images, labels = Variable(images), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

print('Finished Training')
        
    # evaluate
eval_acc = 0.
eval1_acc = 0.
for epoch in range(NUM_EPOCHS):
    model.eval()

    for images, labels in tqdm(train_loader):
        images, labels = Variable(images), Variable(labels)
        out = model(images)
        pred = torch.max(out, 1)[1]
        num_correct = (pred == labels).sum()
        eval_acc += num_correct.data[0]
        
for epoch in range(NUM_EPOCHS):
    model.eval()
    for images, labels in tqdm(test_loader):
        images, labels = Variable(images), Variable(labels)
        out = model(images)
        pred = torch.max(out, 1)[1]
        num_correct = (pred == labels).sum()
        eval1_acc += num_correct.data[0]
    


  1%|          | 3/468 [00:00<00:17, 26.60it/s]

Finished Training


100%|██████████| 78/78 [00:02<00:00, 37.89it/s]


Q5:
Please print the training and testing accuracy.

In [11]:
print('Training ACCURACY:%.3f'%(100.*eval_acc/len(train_dataset)/10))
print('Testing ACCURACY:%.3f'%(100.*eval1_acc/len(test_dataset)/10))

Training ACCURACY:98.000
Testing ACCURACY:98.000
